# Análisis descriptivo - CineVision Studios
Aquí se contiene la resolución del ejercicio #4 y #5 con las consultas correspondientes.

In [50]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

sns.set_theme(style="whitegrid")

file_path = "./movies.csv"
df = pd.read_csv(file_path, encoding="ISO-8859-1")